In [1]:
!pip install gradio
!pip install pdfplumber
!pip install httpx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.5 MB/s eta

In [2]:
import os
import tensorflow as tf
import gradio as gr
import pdfplumber
import networkx as nx
import random
import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt
from matplotlib.patches import Wedge

In [3]:
#Clase Materia
class Materia:
    def __init__(self, codigo, nombre, dificultad, horas_clase, horas_estudio, horas_practica, rama, año):
        self.codigo = codigo
        self.nombre = nombre
        self.dificultad = dificultad
        self.horas_clase = horas_clase
        self.horas_estudio = horas_estudio
        self.horas_practica = horas_practica
        self.rama = rama
        self.año = año

In [4]:
#carga del modelo
if not os.path.isfile("materias.keras"):
#descargar el modelo preentrenado previamente subido a drive
  !gdown 1obTcSz-aWJGGx7XoZ1n2mDYhqsWaLGSz
modelo = tf.keras.models.load_model('materias.keras')

Downloading...
From: https://drive.google.com/uc?id=1obTcSz-aWJGGx7XoZ1n2mDYhqsWaLGSz
To: /content/materias.keras
100% 214k/214k [00:00<00:00, 90.9MB/s]


In [5]:
#Mapa de Materias
mapa_materias = {
    "3621": {"nombre": "Matemática Discreta", "dificultad": 4, "horas_clase": 4, "horas_estudio": 2, "horas_practica": 2, "rama": "Ciencias Básicas", "año": "1"},
    "3622": {"nombre": "Análisis Matemático I", "dificultad": 8, "horas_clase": 4, "horas_estudio": 4, "horas_practica": 5, "rama": "Ciencias Básicas", "año": "1"},
    "3623": {"nombre": "Programación Inicial", "dificultad": 6, "horas_clase": 4, "horas_estudio": 1, "horas_practica": 4, "rama": "Programación", "año": "1"},
    "3624": {"nombre": "Introducción a los Sistemas de Información", "dificultad": 4, "horas_clase": 4, "horas_estudio": 2, "horas_practica": 2, "rama": "Desarrollo de SW", "año": "1"},
    "3625": {"nombre": "Sistemas de Numeración", "dificultad": 5, "horas_clase": 4, "horas_estudio": 2, "horas_practica": 2, "rama": "Infraestructura", "año": "1"},
    "3626": {"nombre": "Principios de Calidad de Software", "dificultad": 2, "horas_clase": 4, "horas_estudio": 2, "horas_practica": 1, "rama": "Calidad y Seguridad de la Información", "año": "1"},
    "3627": {"nombre": "Álgebra y Geometría Analítica I", "dificultad": 7, "horas_clase": 4, "horas_estudio": 3, "horas_practica": 3, "rama": "Ciencias Básicas", "año": "1"},
    "3628": {"nombre": "Física I", "dificultad": 8, "horas_clase": 4, "horas_estudio": 3, "horas_practica": 4, "rama": "Ciencias Básicas", "año": "1"},
    "3629": {"nombre": "Programación Estructurada Básica", "dificultad": 6, "horas_clase": 4, "horas_estudio": 1, "horas_practica": 4, "rama": "Programación", "año": "1"},
    "3630": {"nombre": "Introducción a la Gestión de Requisitos", "dificultad": 5, "horas_clase": 4, "horas_estudio": 2, "horas_practica": 2, "rama": "Desarrollo de SW", "año": "1"},
    "3631": {"nombre": "Fundamentos de Sistemas Embebidos", "dificultad": 5, "horas_clase": 4, "horas_estudio": 2, "horas_practica": 2, "rama": "Infraestructura", "año": "1"},
    "3632": {"nombre": "Introducción a los Proyectos Informáticos", "dificultad": 4, "horas_clase": 4, "horas_estudio": 2, "horas_practica": 1, "rama": "Gestión y Complementarias", "año": "1"},
    "3633": {"nombre": "Análisis Matemático II", "dificultad": 9, "horas_clase": 4, "horas_estudio": 4, "horas_practica": 5, "rama": "Ciencias Básicas", "año": "2"},
    "3634": {"nombre": "Física II", "dificultad": 9, "horas_clase": 4, "horas_estudio": 4, "horas_practica": 4, "rama": "Ciencias Básicas", "año": "2"},
    "3635": {"nombre": "Tópicos de Programación", "dificultad": 7, "horas_clase": 4, "horas_estudio": 2, "horas_practica": 4, "rama": "Programación", "año": "2"},
    "3636": {"nombre": "Bases de Datos", "dificultad": 7, "horas_clase": 4, "horas_estudio": 2, "horas_practica": 4, "rama": "Programación", "año": "2"},
    "3637": {"nombre": "Análisis de Sistemas", "dificultad": 6, "horas_clase": 4, "horas_estudio": 2, "horas_practica": 3, "rama": "Desarrollo de SW", "año": "2"},
    "3638": {"nombre": "Arquitectura de Computadoras", "dificultad": 6, "horas_clase": 4, "horas_estudio": 3, "horas_practica": 3, "rama": "Infraestructura", "año": "2"},
    "3676": {"nombre": "Responsabilidad Social Universitaria", "dificultad": 2, "horas_clase": 4, "horas_estudio": 1, "horas_practica": 1, "rama": "Otras", "año": "2"},
    "3639": {"nombre": "Análisis Matemático III", "dificultad": 6, "horas_clase": 4, "horas_estudio": 3, "horas_practica": 3, "rama": "Ciencias Básicas", "año": "2"},
    "3640": {"nombre": "Algoritmos y Estructuras de Datos", "dificultad": 8, "horas_clase": 4, "horas_estudio": 2, "horas_practica": 4, "rama": "Programación", "año": "2"},
    "3641": {"nombre": "Bases de Datos Aplicadas", "dificultad": 7, "horas_clase": 4, "horas_estudio": 3, "horas_practica": 4, "rama": "Programación", "año": "2"},
    "3642": {"nombre": "Principios de Diseño de Sistemas", "dificultad": 7, "horas_clase": 4, "horas_estudio": 3, "horas_practica": 3, "rama": "Desarrollo de SW", "año": "2"},
    "3643": {"nombre": "Redes de Computadoras", "dificultad": 7, "horas_clase": 4, "horas_estudio": 2, "horas_practica": 3, "rama": "Infraestructura", "año": "2"},
    "3644": {"nombre": "Gestión de las Organizaciones", "dificultad": 4, "horas_clase": 4, "horas_estudio": 6, "horas_practica": 1, "rama": "Gestión y Complementarias", "año": "2"},
    "3680": {"nombre": "Taller de Integración", "dificultad": 4, "horas_clase": 4, "horas_estudio": 2, "horas_practica": 2, "rama": "Otras", "año": "2"},
    "3645": {"nombre": "Álgebra y Geometría Analítica II", "dificultad": 6, "horas_clase": 4, "horas_estudio": 2, "horas_practica": 3, "rama": "Ciencias Básicas", "año": "3"},
    "3646": {"nombre": "Paradigmas de Programación", "dificultad": 8, "horas_clase": 4, "horas_estudio": 2, "horas_practica": 5, "rama": "Programación", "año": "3"},
    "3647": {"nombre": "Requisitos Avanzados", "dificultad": 5, "horas_clase": 4, "horas_estudio": 1, "horas_practica": 4, "rama": "Desarrollo de SW", "año": "3"},
    "3648": {"nombre": "Diseño de Software", "dificultad": 9, "horas_clase": 4, "horas_estudio": 4, "horas_practica": 4, "rama": "Desarrollo de SW", "año": "3"},
    "3649": {"nombre": "Sistemas Operativos", "dificultad": 8, "horas_clase": 4, "horas_estudio": 4, "horas_practica": 3, "rama": "Infraestructura", "año": "3"},
    "3650": {"nombre": "Seguridad de la Información", "dificultad": 4, "horas_clase": 4, "horas_estudio": 3, "horas_practica": 1, "rama": "Calidad y Seguridad de la Información", "año": "3"},
    "3675": {"nombre": "Práctica Profesional Supervisada", "dificultad": 1, "horas_clase": 4, "horas_estudio": 1, "horas_practica": 1, "rama": "Otras", "año": "3"},
    "3651": {"nombre": "Probabilidad y Estadística", "dificultad": 7, "horas_clase": 4, "horas_estudio": 3, "horas_practica": 3, "rama": "Ciencias Básicas", "año": "3"},
    "3652": {"nombre": "Programación Avanzada", "dificultad": 8, "horas_clase": 4, "horas_estudio": 2, "horas_practica": 4, "rama": "Programación", "año": "3"},
    "3653": {"nombre": "Arquitectura de Sistemas Software", "dificultad": 6, "horas_clase": 4, "horas_estudio": 3, "horas_practica": 3, "rama": "Desarrollo de SW", "año": "3"},
    "3654": {"nombre": "Virtualización de Hardware", "dificultad": 9, "horas_clase": 4, "horas_estudio": 2, "horas_practica": 6, "rama": "Infraestructura", "año": "3"},
    "3655": {"nombre": "Auditoria y Legislación", "dificultad": 4, "horas_clase": 4, "horas_estudio": 3, "horas_practica": 1, "rama": "Calidad y Seguridad de la Información", "año": "3"},
    "3656": {"nombre": "Estadística Aplicada", "dificultad": 7, "horas_clase": 4, "horas_estudio": 3, "horas_practica": 3, "rama": "Ciencias Básicas", "año": "4"},
    "3657": {"nombre": "Autómatas y Gramáticas", "dificultad": 7, "horas_clase": 4, "horas_estudio": 3, "horas_practica": 3, "rama": "Programación", "año": "4"},
    "3658": {"nombre": "Programación Concurrente", "dificultad": 8, "horas_clase": 4, "horas_estudio": 3, "horas_practica": 4, "rama": "Infraestructura", "año": "4"},
    "3659": {"nombre": "Gestión Aplic. al Des. de Software I", "dificultad": 6, "horas_clase": 4, "horas_estudio": 2, "horas_practica": 3, "rama": "Desarrollo de SW", "año": "4"},
    "3660": {"nombre": "Sistemas Operativos Avanzados", "dificultad": 8, "horas_clase": 4, "horas_estudio": 3, "horas_practica": 4, "rama": "Infraestructura", "año": "4"},
    "3661": {"nombre": "Gestión de Proyectos", "dificultad": 6, "horas_clase": 4, "horas_estudio": 2, "horas_practica": 4, "rama": "Gestión y Complementarias", "año": "4"},
    "3662": {"nombre": "Matemática Aplicada", "dificultad": 7, "horas_clase": 4, "horas_estudio": 2, "horas_practica": 3, "rama": "Ciencias Básicas", "año": "4"},
    "3663": {"nombre": "Lenguajes y Compiladores", "dificultad": 8, "horas_clase": 4, "horas_estudio": 2, "horas_practica": 4, "rama": "Programación", "año": "4"},
    "3664": {"nombre": "Inteligencia Artificial", "dificultad": 6, "horas_clase": 4, "horas_estudio": 4, "horas_practica": 1, "rama": "Desarrollo de SW", "año": "4"},
    "3665": {"nombre": "Gestión Aplicada al Desarrollo de Software II", "dificultad": 6, "horas_clase": 4, "horas_estudio": 2, "horas_practica": 3, "rama": "Desarrollo de SW", "año": "4"},
    "3666": {"nombre": "Seguridad Aplicada y Forensia", "dificultad": 4, "horas_clase": 4, "horas_estudio": 3, "horas_practica": 1, "rama": "Calidad y Seguridad de la Información", "año": "4"},
    "3667": {"nombre": "Gestión de la Calidad en Procesos de Sistemas", "dificultad": 6, "horas_clase": 4, "horas_estudio": 2, "horas_practica": 3, "rama": "Calidad y Seguridad de la Información", "año": "4"},
    "3668": {"nombre": "Inteligencia Artificial Aplicada", "dificultad": 7, "horas_clase": 4, "horas_estudio": 2, "horas_practica": 4, "rama": "Desarrollo de SW", "año": "5"},
    "3669": {"nombre": "Innovacion y Emprendedorismo", "dificultad": 6, "horas_clase": 4, "horas_estudio": 3, "horas_practica": 2, "rama": "Gestión y Complementarias", "año": "5"},
    "3670": {"nombre": "Ciencia de Datos", "dificultad": 8, "horas_clase": 4, "horas_estudio": 3, "horas_practica": 3, "rama": "Desarrollo de SW", "año": "5"},
    "3671": {"nombre": "Proyecto Final de Carrera", "dificultad": 10, "horas_clase": 4, "horas_estudio": 1, "horas_practica": 8, "rama": "Desarrollo de SW", "año": "5"},
    "3672": {"nombre": "Electiva I", "dificultad": 6, "horas_clase": 4, "horas_estudio": 2, "horas_practica": 2, "rama": "Desarrollo de SW", "año": "5"},
    "3673": {"nombre": "Electiva II", "dificultad": 7, "horas_clase": 4, "horas_estudio": 3, "horas_practica": 2, "rama": "Desarrollo de SW", "año": "5"},
    "3674": {"nombre": "Electiva III", "dificultad": 6, "horas_clase": 4, "horas_estudio": 2, "horas_practica": 3, "rama": "Desarrollo de SW", "año": "5"},
    "3677": {"nombre": "Lenguaje Orientado a Negocios", "dificultad": 6, "horas_clase": 4, "horas_estudio": 2, "horas_practica": 2, "rama": "Desarrollo de SW", "año": "5"},
    "3678": {"nombre": "Tecnologías en Seguridad", "dificultad": 7, "horas_clase": 4, "horas_estudio": 3, "horas_practica": 2, "rama": "Desarrollo de SW", "año": "5"},
    "3679": {"nombre": "Visión Artificial", "dificultad": 6, "horas_clase": 4, "horas_estudio": 2, "horas_practica": 3, "rama": "Desarrollo de SW", "año": "5"},
    "901": {"nombre": "Inglés Transversal Nivel I", "dificultad": 1, "horas_clase": 4, "horas_estudio": 2, "horas_practica": 2, "rama": "Transversal", "año": "T"},
    "902": {"nombre": "Inglés Transversal Nivel II", "dificultad": 3, "horas_clase": 4, "horas_estudio": 2, "horas_practica": 2, "rama": "Transversal", "año": "T"},
    "903": {"nombre": "Inglés Transversal Nivel III", "dificultad": 2, "horas_clase": 4, "horas_estudio": 2, "horas_practica": 2, "rama": "Transversal", "año": "T"},
    "904": {"nombre": "Inglés Transversal Nivel IV", "dificultad": 2, "horas_clase": 4, "horas_estudio": 2, "horas_practica": 2, "rama": "Transversal", "año": "T"},
    "911": {"nombre": "Computación Transversal Nivel I", "dificultad": 2, "horas_clase": 4, "horas_estudio": 1, "horas_practica": 3, "rama": "Transversal", "año": "T"},
    "912": {"nombre": "Computación Transversal Nivel II", "dificultad": 3, "horas_clase": 4, "horas_estudio": 1, "horas_practica": 3, "rama": "Transversal", "año": "T"}
}

In [6]:
# Creamos un grafo dirigido
grafo_correlatividades = nx.DiGraph()

# Lista de materias con sus datos (Codigo Materia, Nombre, Correlativas)
materias_datos = [
    ("3621", "Matemática Discreta", "---"),
    ("3622", "Análisis Matemático I", "---"),
    ("3623", "Programación Inicial", "---"),
    ("3624", "Introducción a los Sistemas de Información", "---"),
    ("3625", "Sistemas de Numeración", "---"),
    ("3626", "Principios de Calidad de Software", "---"),
    ("3627", "Álgebra y Geometría Analítica I", "---"),
    ("3628", "Física I", "3622"),
    ("3629", "Programación Estructurada Básica", "3623"),
    ("3630", "Introducción a la Gestión de Requisitos", "3624"),
    ("3631", "Fundamentos de Sistemas Embebidos", "3625"),
    ("3632", "Introducción a los Proyectos Informáticos", "---"),
    ("3633", "Análisis Matemático II", "3622"),
    ("3634", "Física II", "3628"),
    ("3635", "Tópicos de Programación", "3629 / 3621"),
    ("3636", "Bases de Datos", "3629 / 3621"),
    ("3637", "Análisis de Sistemas", "3630"),
    ("3638", "Arquitectura de Computadoras", "3631"),
    ("3676", "Responsabilidad Social Universitaria", "3626"),
    ("3639", "Análisis Matemático III", "3633"),
    ("3640", "Algoritmos y Estructuras de Datos", "3635"),
    ("3641", "Bases de Datos Aplicadas", "3636"),
    ("3642", "Principios de Diseño de Sistemas", "3637 / 3626"),
    ("3643", "Redes de Computadoras", "3638 / 3634"),
    ("3644", "Gestión de las Organizaciones", "3632"),
    ("3680", "Taller de Integración", "3638 / 3636 / 3635 / 3632 / 3630 / 3626 / 3625 / 3624 / 3623 / 3621"),
    ("3645", "Álgebra y Geometría Analítica II", "3627"),
    ("3646", "Paradigmas de Programación", "3640 / 3633"),
    ("3647", "Requisitos Avanzados", "3642"),
    ("3648", "Diseño de Software", "3642 / 3636"),
    ("3649", "Sistemas Operativos", "3638"),
    ("3650", "Seguridad de la Información", "3643 / 3638 / 3635"),
    ("3675", "Práctica Profesional Supervisada", "3642"),
    ("3651", "Probabilidad y Estadística", "3645 / 3639 / 3621"),
    ("3652", "Programación Avanzada", "3641 / 3646"),
    ("3653", "Arquitectura de Sistemas Software", "3648"),
    ("3654", "Virtualización de Hardware", "3649 / 3645 / 3640"),
    ("3655", "Auditoria y Legislación", "3650"),
    ("3656", "Estadística Aplicada", "3651 / 3641"),
    ("3657", "Autómatas y Gramáticas", "3646"),
    ("3658", "Programación Concurrente", "3654 / 3646"),
    ("3659", "Gestión Aplic. al Des. de Software I", "3653 / 3647 / 3644"),
    ("3660", "Sistemas Operativos Avanzados", "3654"),
    ("3661", "Gestión de Proyectos", "3651 / 3650 / 3644"),
    ("3662", "Matemática Aplicada", "3651"),
    ("3663", "Lenguajes y Compiladores", "3657"),
    ("3664", "Inteligencia Artificial", "3651 / 3646"),
    ("3665", "Gestión Aplicada al Desarrollo de Software II", "3659 / 3652"),
    ("3666", "Seguridad Aplicada y Forensia", "3655 / 3652 / 3649"),
    ("3667", "Gestión de la Calidad en Procesos de Sistemas", "3647"),
    ("3668", "Inteligencia Artificial Aplicada", "3664 / 3656"),
    ("3669", "Innovacion y Emprendedorismo", "3661"),
    ("3670", "Ciencia de Datos", "3664 / 3656"),
    ("3671", "Proyecto Final de Carrera", "3667 / 3661 / 3660 / 3659 / 3656"),
    ("3672", "Electiva I", "3658 / 3661 / 3663"),
    ("3673", "Electiva II", "3662 / 3666"),
    ("3674", "Electiva III", "3664 / 3665"),
    ("3677", "Lenguaje Orientado a Negocios", "3658 / 3661 / 3663"),
    ("3678", "Tecnologías en Seguridad", "3662 / 3666"),
    ("3679", "Visión Artificial", "3664 / 3665"),
    ("901", "Inglés Transversal Nivel I", "---"),
    ("902", "Inglés Transversal Nivel II", "901"),
    ("903", "Inglés Transversal Nivel III", "902"),
    ("904", "Inglés Transversal Nivel IV", "903"),
    ("911", "Computación Transversal Nivel I", "---"),
    ("912", "Computación Transversal Nivel II", "911")
]

# Creamos nodos y arcos en el grafo según las correlatividades
for codigo, nombre, correlatividad in materias_datos:
    grafo_correlatividades.add_node(codigo)
    if correlatividad != "---":
        correlativas = correlatividad.split("/")
        for correlativa in correlativas:
            grafo_correlatividades.add_edge(correlativa.strip(), codigo)  # Aplicamos strip() al código de correlatividad

In [7]:
# Materias inmediatas que se desbloquean (en el proximo cuatrimeste, en caso de aprobar la cursada actual)
def contar_materias_inmediatas(materia_aprobada, grafo):
    materias_inmediatas = set()
    if materia_aprobada in grafo:
        sucesores = grafo.successors(materia_aprobada)
        for sucesor in sucesores:
            materias_inmediatas.add(sucesor)
    return len(materias_inmediatas)

In [8]:
# Función para contar todas las materias desbloqueadas por un historial de materias aprobadas
def contar_todas_materias_desbloqueadas(historial_alumno, grafo):
    materias_desbloqueadas = set()
    for materia_aprobada in historial_alumno:
        if materia_aprobada in grafo:
            materias_desbloqueadas.update(nx.descendants(grafo, materia_aprobada))
    return len(materias_desbloqueadas)

In [9]:
# Funcion para validar si se puede cursar una materia
def todas_correlativas_aprobadas(materia, materias_aprobadas):
    ancestros_directos = grafo_correlatividades.predecessors(materia) #Con predecessors solo evaluas los padres directos
    return all(correlativa in materias_aprobadas for correlativa in ancestros_directos)

In [10]:
horarios = {
    "Lunes-Mañana": None,
    "Lunes-Tarde": None,
    "Lunes-Noche": None,
    "Martes-Mañana": None,
    "Martes-Tarde": None,
    "Martes-Noche": None,
    "Miércoles-Mañana": None,
    "Miércoles-Tarde": None,
    "Miércoles-Noche": None,
    "Jueves-Mañana": None,
    "Jueves-Tarde": None,
    "Jueves-Noche": None,
    "Viernes-Mañana": None,
    "Viernes-Tarde": None,
    "Viernes-Noche": None,
    "Sábado-Mañana": None,
    "Sábado-Tarde": None,
    "Sábado-Noche": None,
}

In [11]:
def extraer_tabla_desde_pdf(ruta_pdf):
    tablas = []
    with pdfplumber.open(ruta_pdf) as pdf:
        for pagina in pdf.pages:
            tabla = pagina.extract_table()
            if tabla:
                tablas.append(tabla)
    return tablas

In [12]:
def extraer_datos_oferta_materias(tablas):
    datos_extraidos = []
    ultimo_codigo = None
    ultima_descripcion = None

    for tabla in tablas:
        for fila in tabla[1:]:  # Ignora la primera fila (encabezados)
            codigo = fila[0] if fila[0] else ultimo_codigo
            descripcion = fila[1] if fila[1] else ultima_descripcion
            codigo_comision = fila[2]
            dias = fila[4].split("\n")[0]  # Extraer el día y el horario
            modalidad = fila[5]
            sede = fila[6]

            # Verifica si el código de comisión está marcado como "No ofertada"
            if "no ofertada" in codigo_comision.lower():
                continue  # Salta a la siguiente fila si no se ofrece la comisión

            # Se obteniene el día y el horario a partir de "Días"
            dia, horario = "", ""
            if dias:
                # Convierte códigos de día a nombres legibles
                codigo_dia = dias[:-5]
                nombres_dia = {"Lu": "Lunes", "Ma": "Martes", "Mi": "Miércoles", "Ju": "Jueves", "Vi": "Viernes", "Sa": "Sábado", "Do": "Domingo"}
                dia = nombres_dia.get(codigo_dia, "")

                # Convierte códigos de turno a nombres legibles
                codigo_turno = dias[-5:]
                nombres_turno = {"08a12": "Mañana", "14a18": "Tarde", "19a23": "Noche"}
                horario = nombres_turno.get(codigo_turno, "")

            entrada = {
                "Codigo de materia": codigo.lstrip('0'),
                "Descripcion": descripcion,
                "Dia": dia,
                "Horario": horario,
                "Modalidad": modalidad,
                "Sede": sede
            }
            datos_extraidos.append(entrada)

            # Actualiza los últimos códigos de materia y descripción leídos
            ultimo_codigo = codigo
            ultima_descripcion = descripcion

    return datos_extraidos

In [13]:
def extraer_materias_aprobadas(pdf_path):
    materias_aprobadas = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            tabla = page.extract_table()
            if tabla:
                for fila in tabla[1:]:  # Ignora la primera fila que generalmente es el encabezado
                    # Extraer los campos de interés
                    codigo_materia = fila[0][1:]  # Eliminar el primer dígito (el 0)
                    nombre_materia = fila[1]
                    fecha = fila[2]
                    nota = fila[3]
                    acta = fila[4]
                    # Agregar los datos a la lista de materias aprobadas
                    materias_aprobadas.append({
                        "codigo_materia": codigo_materia.lstrip('0'),
                        "nombre_materia": nombre_materia,
                        "fecha": fecha,
                        "nota": nota,
                        "acta": acta
                    })
    return materias_aprobadas

In [14]:
# Devuelve una lista con las materias que el usuario puede cursar
def obtener_materias_cursables(mapa_materias, grafo_correlatividades, materias_aprobadas_usuario):
    materias_cursables = []
    for codigo_materia, materia in mapa_materias.items():
        if codigo_materia in materias_aprobadas_usuario:
            continue  # Si la materia ya fue aprobada, pasa a la siguiente

        # Verifica si todas las materias predecesoras están aprobadas
        predecesoras_aprobadas = todas_correlativas_aprobadas(codigo_materia, materias_aprobadas_usuario)
        if predecesoras_aprobadas:
            materias_cursables.append(codigo_materia)

    return materias_cursables

In [15]:
# Funcion que devuelve una lista con las materias aprobadas por el alumno
def obtener_materias_aprobadas_por_usuario(mapa_materias, materias_aprobadas):
    materias_aprobadas_usuario = []
    for materia in materias_aprobadas:
        codigo_materia = materia["codigo_materia"]
        if codigo_materia in mapa_materias:
            materias_aprobadas_usuario.append(codigo_materia)
    return materias_aprobadas_usuario

In [16]:
# En base a las materias que puede cursar, devuelve todos los horarios posibles para esas materias
def obtener_horarios_disponibles(oferta_de_materias, materias_cursables):
    horarios_disponibles = {}
    for oferta in oferta_de_materias:
        codigo_oferta = oferta['Codigo de materia']
        if codigo_oferta in materias_cursables:
            materia = codigo_oferta
            dia = oferta['Dia']
            horario = oferta['Horario']
            if materia not in horarios_disponibles:
                horarios_disponibles[materia] = []
            horarios_disponibles[materia].append((dia, horario))
    return horarios_disponibles

In [17]:
def inicializar_poblacion(horarios_disponibles, num_individuos):
    poblacion = []
    min_materias = 1
    max_materias = len(list(horarios_disponibles.keys()))

    for _ in range(num_individuos):
        # Creamos un nuevo individuo
        horario_individuo = {}
        materias_asignadas = set()  # Para verificar si una materia ya ha sido asignada
        dias_horarios_asignados = set()  # Para verificar si un par dia-horario ya ha sido asignado

        # Determina aleatoriamente el número de materias para este individuo
        num_materias = random.randint(min_materias, max_materias)

        # Lista de materias disponibles que pueden ser seleccionadas
        materias_disponibles = list(horarios_disponibles.keys())

        # Bandera para verificar si se rechazaron todos los horarios de todas las materias
        todas_rechazadas = False

        # Seleccionamos materias aleatorias y las asignamos a días y horarios
        while len(horario_individuo) < num_materias:
            # Seleccionamos una materia aleatoria disponible
            materia = random.choice(materias_disponibles)

            # Verificamos si todos los horarios de la materia ya han sido rechazados
            horarios_materia = horarios_disponibles[materia]
            if all(horario in dias_horarios_asignados for horario in horarios_materia):
                # Si todos los horarios de la materia ya han sido rechazados, la eliminamos de las disponibles
                materias_disponibles.remove(materia)
                # Si no quedan materias disponibles, terminamos la inicialización del individuo
                if not materias_disponibles:
                    todas_rechazadas = True
                    break
                continue  # Pasamos a la siguiente iteración del bucle

            # Seleccionamos un día y horario aleatorio para la materia
            dia_horario = random.choice(horarios_materia)

            # Verificamos si el día y horario ya han sido asignados
            if dia_horario in dias_horarios_asignados:
                # Si el horario ya está ocupado, pasamos a la siguiente iteración del bucle
                continue

            # Asignamos la materia al día y horario seleccionados
            horario_individuo[materia] = dia_horario

            # Agregamos la materia y el día/horario a los conjuntos de materias y días/horarios asignados
            materias_asignadas.add(materia)
            dias_horarios_asignados.add(dia_horario)

        if todas_rechazadas:
            poblacion.append(horario_individuo)
            continue  # Si se rechazaron todos los horarios de todas las materias, terminamos la inicialización del individuo

        poblacion.append(horario_individuo)

    return poblacion

In [18]:
def evaluar_exito(materias, trabaja, horas_estudio):
    # Verifico si la lista de materias tiene menos de 6 elementos
    if len(materias) < 6:
        # Relleno materias con 0 hasta tener 6 elementos. La red neuronal esta pensada para analizar 6 materias
        materias += [0] * (6 - len(materias))

    #evito usar el for para no añadir complejidad innecesaria
    sample = {"M1":materias[0],"D1":mapa_materias[str(materias[0])]["dificultad"] if materias[0] != 0 else 0,
              "HC1":mapa_materias[str(materias[0])]["horas_clase"] if materias[0] != 0 else 0,
              "HE1":mapa_materias[str(materias[0])]["horas_estudio"] if materias[0] != 0 else 0,
              "HP1":mapa_materias[str(materias[0])]["horas_practica"] if materias[0] != 0 else 0,
              "M2":materias[1],"D2":mapa_materias[str(materias[1])]["dificultad"] if materias[1] != 0 else 0,
              "HC2":mapa_materias[str(materias[1])]["horas_clase"] if materias[1] != 0 else 0,
              "HE2":mapa_materias[str(materias[1])]["horas_estudio"] if materias[1] != 0 else 0,
              "HP2":mapa_materias[str(materias[1])]["horas_practica"] if materias[1] != 0 else 0,
              "M3":materias[2],"D3":mapa_materias[str(materias[2])]["dificultad"] if materias[2] != 0 else 0,
              "HC3":mapa_materias[str(materias[2])]["horas_clase"] if materias[2] != 0 else 0,
              "HE3":mapa_materias[str(materias[2])]["horas_estudio"] if materias[2] != 0 else 0,
              "HP3":mapa_materias[str(materias[2])]["horas_practica"] if materias[2] != 0 else 0,
              "M4":materias[3],"D4":mapa_materias[str(materias[3])]["dificultad"] if materias[3] != 0 else 0,
              "HC4":mapa_materias[str(materias[3])]["horas_clase"] if materias[3] != 0 else 0,
              "HE4":mapa_materias[str(materias[3])]["horas_estudio"] if materias[3] != 0 else 0,
              "HP4":mapa_materias[str(materias[3])]["horas_practica"] if materias[3] != 0 else 0,
              "M5":materias[4],"D5":mapa_materias[str(materias[4])]["dificultad"] if materias[4] != 0 else 0,
              "HC5":mapa_materias[str(materias[4])]["horas_clase"] if materias[4] != 0 else 0,
              "HE5":mapa_materias[str(materias[4])]["horas_estudio"] if materias[4] != 0 else 0,
              "HP5":mapa_materias[str(materias[4])]["horas_practica"] if materias[4] != 0 else 0,
              "M6":materias[5],"D6":mapa_materias[str(materias[5])]["dificultad"] if materias[5] != 0 else 0,
              "HC6":mapa_materias[str(materias[5])]["horas_clase"] if materias[5] != 0 else 0,
              "HE6":mapa_materias[str(materias[5])]["horas_estudio"] if materias[5] != 0 else 0,
              "HP6":mapa_materias[str(materias[5])]["horas_practica"] if materias[5] != 0 else 0,
              "Trabaja": trabaja, "Horas_Estudio_Semanales": horas_estudio}

    input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}
    #evaluo usando __call()__
    #predict() es menos performante en este caso particular (diferencia de 10 min a 30 segundos)
    #https://www.tensorflow.org/api_docs/python/tf/keras/Model#predict
    predictions = modelo(input_dict)
    prob = tf.nn.sigmoid(predictions[0])

    return prob.numpy()[0] * 100

In [19]:
def calcular_aptitud(individuo, mapa_materias, informacion_usuario, grafo_correlatividades):
  cant_materias = len(individuo)
  #Si el individuo no tiene materias, se le pone la minima aptitud
  if cant_materias == 0:
      return 0

  # Parametro 1: Probabilidad de Exito de la cursada (tiene en cuenta difcultad de la cursada y las horas semanales de estudio)
  materias_individuo = [int(key) for key in individuo.keys()] # Si no lo casteo a int, lo toma como string y falla la red
  if informacion_usuario['trabaja'] == True:
    trabaja = 1
  else:
    trabaja = 0
  probabilidad_exito = evaluar_exito(materias_individuo, trabaja, informacion_usuario['horas_estudio'])

  # Parametro 2: Dificultad de la cursada (se penaliza si es muy facil o muy dificil)
  suma_dificultad = 0
  for materia, horario in individuo.items():
     info_materia = mapa_materias[materia]
     suma_dificultad += info_materia["dificultad"]

  promedio_dificultad = suma_dificultad / cant_materias

  penalizacion_dificultad = 0

  if promedio_dificultad > 7:
        penalizacion_dificultad = (promedio_dificultad - 7) * 10  # Cuatrimestre muy dificil. Por cada punto de dificultad arriba de 7 resta 5 de aptitud

  if promedio_dificultad < 4:
        penalizacion_dificultad += (4 - promedio_dificultad) * 10 # Cuatrimestre muy facil. Por cada punto de dificultad menor a 4 resta 5 de aptitud

  # Parametro 3: Cantidad de horas semanales requeridas
  suma_horas_semanales = 0
  for materia, horario in individuo.items():
    info_materia = mapa_materias[materia]
    suma_horas_semanales += info_materia["horas_clase"] + info_materia["horas_estudio"] + info_materia["horas_practica"]

  horas_estudio_limite = informacion_usuario['horas_estudio']
  porcentaje_diferencia = abs(suma_horas_semanales - horas_estudio_limite) / horas_estudio_limite

  penalizacion_horas = 0
  if porcentaje_diferencia > 0.1:
    penalizacion_horas = porcentaje_diferencia * 50  # Diferencia entre horas estimadas / horas reales. Cada 10% de diferencia resta 5 de aptitud

  # Parametro 4: Materias que desbloquea instantaneamente
  cant_materias_inmediatas = 0
  cant_materias_inmediatas += contar_materias_inmediatas(materia, grafo_correlatividades)

  premio_materias_instantaneas_desbloqueadas = cant_materias_inmediatas * 2 # Por cada materia instantanea que desbloquea suma 2 de aptitud

  # Parametro 5: Materias que desbloquea hasta el final de la carrera
  codigos_de_materias = individuo.keys()
  cant_materias_desbloqueadas = 0
  for materia in codigos_de_materias:
    cant_materias_desbloqueadas += contar_todas_materias_desbloqueadas(materia, grafo_correlatividades)

  premio_materias_totales_desbloqueadas = cant_materias_desbloqueadas * 4 # Por cada materia final que desbloquea suma

  # Parametro 6: Cantidad de Materias de los primeros años para el titulo intermedio
  cantidad_materias_primero = 0
  cantidad_materias_segundo = 0
  cantidad_materias_tercero = 0
  cantidad_materias_cuarto = 0
  cantidad_materias_quinto = 0
  cantidad_materias_transversal = 0
  premio_año = 0

  for codigo in codigos_de_materias:
    año = mapa_materias[codigo]['año']

    if año == 1:
      premio_año += 5

    if año == 2:
      premio_año += 4

    if año == 3:
      premio_año += 3

    if año == 4:
      premio_año += 2

    if año == 5:
      premio_año += 1

    if año == 'T':
      premio_año += 3

  # Calculo la aptitud con los cinco parametros
  aptitud = probabilidad_exito - penalizacion_horas - penalizacion_dificultad + premio_materias_instantaneas_desbloqueadas + premio_materias_totales_desbloqueadas + premio_año

  # Normalizo la aptitud con la funcion sigmoide para trabajar solo con aptitudes entre 0 y 1
  aptitud_normalizada = 1 / (1 + np.exp(-aptitud))

  return aptitud

In [20]:
# Diccionario para almacenar las aptitudes calculadas
aptitudes_calculadas = {}

# Función para generar una clave única para una combinación de materias
def generar_clave_individuo(individuo):
    # Extrae solo los códigos de materia de la combinación y los ordena
    codigos_materias = sorted(individuo.keys())
    # Convierte la lista de códigos de materia en una tupla para garantizar la inmutabilidad
    clave_individuo = tuple(codigos_materias)
    return clave_individuo

# Función para calcular la aptitud de una combinación y almacenarla en el diccionario
def calcular_y_almacenar_aptitud(individuo, mapa_materias, informacion_usuario, grafo_correlatividades):
    clave_individuo = generar_clave_individuo(individuo)
    if clave_individuo not in aptitudes_calculadas:
        aptitud = calcular_aptitud(individuo, mapa_materias, informacion_usuario, grafo_correlatividades)
        aptitudes_calculadas[clave_individuo] = aptitud
    return aptitudes_calculadas[clave_individuo]

In [21]:
def seleccion_por_torneo(poblacion, mapa_materias, informacion_usuario, grafo_correlatividades, num_seleccionados):
    # Copio la población para trabajar con ella
    poblacion_copia = deepcopy(poblacion)

    seleccionados = []
    while len(seleccionados) < num_seleccionados:
        # Selecciono dos individuos aleatorios de la copia de la población
        individuo1 = random.choice(poblacion_copia)
        poblacion_copia.remove(individuo1)  # Elimino el individuo seleccionado para no repetirlo en otro torneo
        individuo2 = random.choice(poblacion_copia)
        poblacion_copia.remove(individuo2)  # Elimino el individuo seleccionado para no repetirlo en otro torneo

        # Calculo la aptitud de cada individuo
        #aptitud_individuo1 = calcular_aptitud(individuo1, mapa_materias, informacion_usuario, grafo_correlatividades)
        #aptitud_individuo2 = calcular_aptitud(individuo2, mapa_materias, informacion_usuario, grafo_correlatividades)
        # Calculo la aptitud de cada individuo optimizado
        aptitud_individuo1 = calcular_y_almacenar_aptitud(individuo1, mapa_materias, informacion_usuario, grafo_correlatividades)
        aptitud_individuo2 = calcular_y_almacenar_aptitud(individuo2, mapa_materias, informacion_usuario, grafo_correlatividades)

        # Elijo al individuo más apto como ganador del torneo
        if aptitud_individuo1 > aptitud_individuo2:
            seleccionados.append(individuo1)
        else:
            seleccionados.append(individuo2)

    # Ordeno los seleccionados por aptitud de mayor a menor
    seleccionados.sort(key=lambda x: calcular_aptitud(x, mapa_materias, informacion_usuario, grafo_correlatividades), reverse=True)

    # Devuelvo los mejores individuos
    return seleccionados[:num_seleccionados]

In [22]:
def cruzar_individuos(padre_a, padre_b):
    hijo_a = {}
    hijo_b = {}

    # Definimos los días de la semana
    dias = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado']

    # Padre A
    for materia, dia_horario in padre_a.items():
        if dia_horario is None:
            hijo_a[materia] = None
        else:
            # Si el día es Lunes, Miércoles o Viernes, asignamos las materias del Padre A al Hijo A
            if dia_horario[0] in ['Lunes', 'Miércoles', 'Viernes']:
                hijo_a[materia] = dia_horario
            # Si el día es Martes, Jueves o Sábado, asignamos las materias del Padre A al Hijo B
            else:
                hijo_b[materia] = dia_horario

    # Padre B
    for materia, dia_horario in padre_b.items():
        if dia_horario is None:
            hijo_b[materia] = None
        else:
            # Si el día es Lunes, Miércoles o Viernes, asignamos las materias del Padre B al Hijo B
            if dia_horario[0] in ['Lunes', 'Miércoles', 'Viernes']:
                hijo_b[materia] = dia_horario
            # Si el día es Martes, Jueves o Sábado, asignamos las materias del Padre B al Hijo A
            else:
                hijo_a[materia] = dia_horario

    # Creamos una lista de las claves de los hijos para iterar
    materias_hijo_a = list(hijo_a.keys())
    materias_hijo_b = list(hijo_b.keys())

    # Verificamos si hay materias repetidas en los hijos
    for materia in materias_hijo_a:
        if materia in hijo_b:
            # Hay una materia repetida, eliminamos uno de los horarios aleatoriamente
            if random.random() < 0.5:
                del hijo_a[materia]
            else:
                del hijo_b[materia]

    return hijo_a, hijo_b

In [23]:
def generar_hijos_seleccion(poblacion_seleccionada):
    poblacion_cruza = []

    # Iteramos sobre los padres seleccionados en pares
    for i in range(0, len(poblacion_seleccionada) - 1, 2):
        # Recorremos toda la poblacion seleccionada y vamos haciendo al cruza secuencialmente
        padre_a = poblacion_seleccionada[i]
        padre_b = poblacion_seleccionada[i+1]

        # Cruzamos los padres y obtenemos los hijos
        hijo_a, hijo_b = cruzar_individuos(padre_a, padre_b)

        # Agregamos los hijos a la población de cruza
        poblacion_cruza.append(hijo_a)
        poblacion_cruza.append(hijo_b)

    return poblacion_cruza

In [24]:
def mutar_individuos(poblacion, horarios_disponibles):
    poblacion_mutada = []

    for individuo in poblacion:
        # Creamos una copia del individuo original
        individuo_mutado = individuo.copy()

        # Determinamos el tipo de mutación que se aplicará
        tipos_mutacion = ["agregar_materia", "intercambiar_horarios", "eliminar_materia", "ninguna"]
        probabilidades = [0.1, 0.1, 0.1, 0.7]  # Ajustamos las probabilidades
        tipo_mutacion = random.choices(tipos_mutacion, weights=probabilidades)[0]

        # Mutación tipo 1: Agregar materia en un horario en blanco
        if tipo_mutacion == "agregar_materia":
          horarios_blancos = [dia_horario for dia_horario, materia in individuo_mutado.items() if not materia]
          if horarios_blancos:
              dia_horario_vacio = random.choice(horarios_blancos)
              # Verifica si la materia seleccionada ya está en otro día-turno
              materia_nueva = None
              for _ in range(len(horarios_disponibles)):
                  materia_nueva = random.choice(list(horarios_disponibles.keys()))
                  materia_existente = False
                  for dia_horario, materia_existente in individuo_mutado.items():
                      if materia_existente == materia_nueva:
                          materia_existente = True
                          break
                  if not materia_existente:
                      break  # Sale del bucle si la materia no está en otro día-turno
              if not materia_existente:
                  individuo_mutado[dia_horario_vacio] = materia_nueva
              else:
                  tipo_mutacion = "ninguna"
          else:
              tipo_mutacion = "ninguna"

        # Mutación tipo 2: Intercambiar horarios
        elif tipo_mutacion == "intercambiar_horarios":
            horarios_ocupados = [dia_horario for dia_horario, materia in individuo_mutado.items() if materia]
            if len(horarios_ocupados) >= 1:
                # Selecciona un horario ocupado aleatoriamente
                horario = random.choice(horarios_ocupados)
                # Selecciona una materia del horario
                materia_a_cambiar = individuo_mutado[horario]
                # Verifica si la materia tiene más de un horario disponible
                horarios_materia = horarios_disponibles.get(materia_a_cambiar, [])
                if len(horarios_materia) > 1:
                    # Selecciona otro horario disponible para la misma materia que no esté ocupado en el individuo
                    horarios_disponibles_otro_horario = [h for h in horarios_materia if h != horario and individuo_mutado.get(h) is None]
                    if horarios_disponibles_otro_horario:
                        otro_horario = random.choice(horarios_disponibles_otro_horario)
                        # Intentar intercambia la materia actual con otra materia del mismo horario que tenga otro horario disponible
                        for otro_dia_horario, otra_materia in individuo_mutado.items():
                            if otro_dia_horario != horario and horarios_disponibles.get(otra_materia, []) and individuo_mutado[otro_dia_horario] != materia_a_cambiar:
                                otros_horarios_materia = [h for h in horarios_disponibles[otra_materia] if h != otro_dia_horario and individuo_mutado.get(h) is None]
                                if otros_horarios_materia:
                                    otro_horario_2 = random.choice(otros_horarios_materia)
                                    # Intercambia las materias si el intercambio es válido
                                    if (otro_dia_horario, materia_a_cambiar) in horarios_disponibles[otra_materia] and (otro_horario_2, otra_materia) in horarios_disponibles[materia_a_cambiar]:
                                        individuo_mutado[horario], individuo_mutado[otro_dia_horario] = otra_materia, materia_a_cambiar
                                        individuo_mutado[otro_horario], individuo_mutado[otro_horario_2] = materia_a_cambiar, otra_materia
                                        break
                # Si no se pudo realizar ningún intercambio, marcar la mutación como "ninguna"
                tipo_mutacion = "ninguna"

        # Mutación tipo 3: Eliminar materia
        elif tipo_mutacion == "eliminar_materia":
            horarios_ocupados = [dia_horario for dia_horario, materia in individuo_mutado.items() if materia]
            if horarios_ocupados:
                horario_a_eliminar = random.choice(horarios_ocupados)
                individuo_mutado[horario_a_eliminar] = None
                del individuo_mutado[str(horario_a_eliminar)] # Elimina la clave asociada a la materia
            else:
                tipo_mutacion = "ninguna"

        # Agregamos el individuo mutado a la población mutada
        poblacion_mutada.append(individuo_mutado)

    return poblacion_mutada

In [25]:
def calcular_aptitud_general(poblacion, mapa_materias, informacion_usuario, grafo_correlatividades):
    aptitud_total = 0
    for individuo in poblacion:
        aptitud_total += calcular_aptitud(individuo, mapa_materias, informacion_usuario, grafo_correlatividades)
    return aptitud_total

In [26]:
# Lista de opciones disponibles en el select

options = [
    ("Matemática Discreta","3621" ),
    ("Análisis Matemático I","3622"),
    ("Programación Inicial","3623"),
    ("Introducción a los Sistemas de Información","3624"),
    ("Sistemas de Numeración","3625" ),
    ("Principios de Calidad de Software","3626"),
    ("Álgebra y Geometría Analítica I","3627"),
    ("Física I","3628"),
    ( "Programación Estructurada Básica","3629"),
    ("Introducción a la Gestión de Requisitos","3630"),
    ("Fundamentos de Sistemas Embebidos","3631"),
    ("Introducción a los Proyectos Informáticos","3632"),
    ("Análisis Matemático II","3633"),
    ("Física II","3634"),
    ( "Tópicos de Programación","3635"),
    ("Bases de Datos","3636"),
    ("Análisis de Sistemas","3637"),
    ("Arquitectura de Computadoras","3638"),
    ("Responsabilidad Social Universitaria","3676"),
    ("Análisis Matemático III","3639"),
    ("Algoritmos y Estructuras de Datos","3640"),
    ("Bases de Datos Aplicadas","3641"),
    ("Principios de Diseño de Sistemas","3642"),
    ( "Redes de Computadoras","3643"),
    ( "Gestión de las Organizaciones","3644"),
    ("Taller de Integración","3680"),
    ("Álgebra y Geometría Analítica II","3645"),
    ("Paradigmas de Programación","3646"),
    ("Requisitos Avanzados","3647"),
    ("Diseño de Software","3648"),
    ("Sistemas Operativos","3649"),
    ("Seguridad de la Información","3650"),
    ("Práctica Profesional Supervisada","3675"),
    ("Probabilidad y Estadística","3651"),
    ( "Programación Avanzada","3652"),
    ( "Arquitectura de Sistemas Software","3653"),
    ( "Virtualización de Hardware","3654"),
    ("Auditoria y Legislación","3655"),
    ("Estadística Aplicada","3656"),
    ("Autómatas y Gramáticas","3657"),
    ("Programación Concurrente","3658"),
    ("Gestión Aplic. al Des. de Software I","3659"),
    ("Sistemas Operativos Avanzados","3660"),
    ("Gestión de Proyectos","3661"),
    ("Matemática Aplicada","3662"),
    ("Lenguajes y Compiladores","3663"),
    ( "Inteligencia Artificial","3664"),
    ("Gestión Aplicada al Desarrollo de Software II","3665"),
    ("Seguridad Aplicada y Forensia","3666"),
    ("Gestión de la Calidad en Procesos de Sistemas","3667"),
    ("Inteligencia Artificial Aplicada","3668"),
    ("Innovacion y Emprendedorismo","3669"),
    ("Ciencia de Datos","3670"),
    ("Proyecto Final de Carrera","3671"),
    ("Electiva I","3672"),
    ("Electiva II","3673"),
    ("Electiva III","3674"),
    ("Inglés Transversal Nivel I","901"),
    ("Inglés Transversal Nivel II","902"),
    ("Inglés Transversal Nivel III","903"),
    ("Inglés Transversal Nivel IV","904"),
    ("Computación Transversal Nivel I","911"),
    ( "Computación Transversal Nivel II","912")
]


In [27]:
def main2(materias_pendiente_final,pdf_oferta,trabaja,horas,ingresante=False,pdf_cursadas=None):


  tablas = extraer_tabla_desde_pdf(pdf_oferta)
  oferta_de_materias = extraer_datos_oferta_materias(tablas)
  if not ingresante:
    materias_aprobadas = extraer_materias_aprobadas(pdf_cursadas)
  else:
    materias_aprobadas = []


  if materias_pendiente_final is not None:
    for mat_pend in materias_pendiente_final:
      materias_aprobadas.append({
                        "codigo_materia": mat_pend,
                        "nombre_materia":'',
                        "fecha":'',
                        "nota":'',
                        "acta":''})


  materias_aprobadas_usuario = obtener_materias_aprobadas_por_usuario(mapa_materias, materias_aprobadas)
  materias_cursables = obtener_materias_cursables(mapa_materias, grafo_correlatividades, materias_aprobadas_usuario)
  horarios_disponibles = obtener_horarios_disponibles(oferta_de_materias, materias_cursables)
  informacion_usuario = {
    "trabaja": trabaja,
    "horas_estudio": horas
}

  num_individuos = 20
  poblacion = inicializar_poblacion(horarios_disponibles, num_individuos)
  aptitud_maxima_historica = 0
  iteraciones_sin_mejora = 0
  max_iteraciones_sin_mejora = 10
  max_iteraciones = 50

  for _ in range(max_iteraciones):
      # Calculo aptitud de la población actual
      aptitud_poblacion = calcular_aptitud_general(poblacion, mapa_materias, informacion_usuario, grafo_correlatividades)

      # Verifico si hay mejora
      if aptitud_poblacion > aptitud_maxima_historica:
          aptitud_maxima_historica = aptitud_poblacion
          iteraciones_sin_mejora = 0
      else:
          iteraciones_sin_mejora += 1

      #Si se alcanza el límite de iteraciones sin mejora, se detiene el algoritmo
      if iteraciones_sin_mejora >= max_iteraciones_sin_mejora:
        break

      # Selecciono la mitad mejor de la población
      poblacion_seleccionada = seleccion_por_torneo(poblacion, mapa_materias, informacion_usuario, grafo_correlatividades, num_seleccionados=len(poblacion) // 2)

      # Genero hijos mediante cruzas
      poblacion_hijos = generar_hijos_seleccion(poblacion_seleccionada)

      # Combino padres e hijos para formar la nueva población
      poblacion_combinada = poblacion_seleccionada + poblacion_hijos

      # Aplico mutación a la población combinada
      poblacion_mutada = mutar_individuos(poblacion_combinada, horarios_disponibles)

      # Selecciono nuevamente la mitad mejor de la población mutada
      poblacion = seleccion_por_torneo(poblacion_mutada, mapa_materias, informacion_usuario, grafo_correlatividades, num_seleccionados=len(poblacion) // 2) + poblacion_seleccionada
      # Obtener el individuo con la mejor aptitud de la población final
  mejor_individuo = max(poblacion, key=lambda individuo: calcular_aptitud(individuo, mapa_materias, informacion_usuario, grafo_correlatividades))

  # Definimos un diccionario para asignar valores numéricos a los turnos
  orden_turnos = {'Mañana': 1, 'Tarde': 2, 'Noche': 3}
  dias_semana = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado']

  # Crear un gráfico
  fig, ax = plt.subplots(figsize=(8, 4))  # Tamaño de la figura

  # Dibujar líneas interiores para separar los horarios
  for i in range(1, 3):
      ax.axhline(i, color='black', linestyle='--', linewidth=0.5)

  # Dibujar líneas interiores para separar los días de la semana
  for i in range(1, 6):
      ax.axvline(i, color='black', linestyle='--', linewidth=0.5)

  # Etiquetar los días de la semana arriba del rectángulo
  dias_semana = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado']
  for i, dia in enumerate(dias_semana):
      ax.text(i + 0.5, 3.15, dia, ha='center')

  # Etiquetar los horarios
  horarios = ['Mañana', 'Tarde', 'Noche']
  for i, horario in enumerate(horarios):
      ax.text(-0.5, 2.5 - i, horario, ha='center', va='center')

  # Colores pastel predefinidos
  colores = [
      '#bc98f3', '#d3bcf6', '#bae0f5', '#d1eaf9', '#d8f79a', '#ff6565', '#ff9c9c', '#9bfae0',
      '#c3f9ea', '#c3dff9', '#ebf3a0', '#f5fac1', '#ba9df4', '#d0bdf6', '#ffda89', '#ffe4e1',
      '#d8f8e1', '#fcb7af', '#b0f2c2', '#b0c2f2', '#fabfb7', '#fdf9c4', '#ffda9e', '#c5c6c8', '#b2e2f2'
  ]

  random.shuffle(colores)

  # Asignar una posición aleatoria para cada materia en el rectángulo
  for codigo_horario, (dia, turno) in mejor_individuo.items():
      nombre_materia = mapa_materias.get(codigo_horario, {}).get("nombre", "Materia Desconocida")
      fila = {'Mañana': 2, 'Tarde': 1, 'Noche': 0}[turno]
      columna = dias_semana.index(dia)
      # Calcular el ancho y alto del rectángulo de la celda
      ancho_celda = 1
      alto_celda = 1
      # Obtener el color de la celda según su posición en la lista
      color_celda = colores.pop(0)
      # Dibujar rectángulos para colorear celdas
      ax.add_patch(plt.Rectangle((columna, fila), ancho_celda, alto_celda, color=color_celda, zorder=0))
      # Agregar el nombre de la materia
      txt = ax.text(columna + 0.5, fila + 0.5, nombre_materia, ha='center', va='center', fontsize=9, wrap=True)
      txt._get_wrap_line_width = lambda : 50

  # Configurar ejes y etiquetas
  ax.set_xlim(0, 6)
  ax.set_ylim(0, 3)
  ax.set_xticks([])
  ax.set_yticks([])

  # Ajustar margen superior para dejar espacio para los días de la semana
  ax.margins(y=0.5)

  # Configurar título
  plt.title('Cursada Recomendada', pad=30, fontsize=15)  # Ajustar el espacio entre el título y el gráfico

  # Mostrar el gráfico
  plt.tight_layout()
  return fig

In [28]:
def evaluar_exito_str(materias, trabaja, horas_estudio):
    sample = dict()

   # return materias
    #Completamos los datos que faltan
   # materias=[3639,3640,3641,3642,0,0]
     # Verifico si la lista de materias tiene menos de 6 elementos
    materias = [int(num) for num in materias]
    if len(materias) < 6:
        # Relleno materias con 0 hasta tener 6 elementos. La red neuronal esta pensada para analizar 6 materias
        materias += [0] * (6 - len(materias))

    sample["Trabaja"] = trabaja
    sample["Horas_Estudio_Semanales"] = horas_estudio

    #Evaluacion utilizando el modelo:
    for n,mat in enumerate(materias):
      sample['M'+str(n+1)]=mat
      sample['D'+str(n+1)]=mapa_materias[str(mat)]["dificultad"] if mat != 0 else 0
      sample['HC'+str(n+1)]=mapa_materias[str(mat)]["horas_clase"] if mat != 0 else 0
      sample['HE'+str(n+1)]=mapa_materias[str(mat)]["horas_estudio"] if mat != 0 else 0
      sample['HP'+str(n+1)]=mapa_materias[str(mat)]["horas_practica"] if mat != 0 else 0

    input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}
    predictions = modelo.predict(input_dict)
    prob = tf.nn.sigmoid(predictions[0])



    return "Probabilidad de exito: %.1f" % (100 * prob)

In [29]:
def evaluar_exito_str2(materias, trabaja, horas_estudio):
    sample = dict()

    materias = [int(num) for num in materias]
    if len(materias) < 6:
        # Relleno materias con 0 hasta tener 6 elementos. La red neuronal esta pensada para analizar 6 materias
        materias += [0] * (6 - len(materias))

    sample["Trabaja"] = trabaja
    sample["Horas_Estudio_Semanales"] = horas_estudio

    #Evaluacion utilizando el modelo:
    for n,mat in enumerate(materias):
      sample['M'+str(n+1)]=mat
      sample['D'+str(n+1)]=mapa_materias[str(mat)]["dificultad"] if mat != 0 else 0
      sample['HC'+str(n+1)]=mapa_materias[str(mat)]["horas_clase"] if mat != 0 else 0
      sample['HE'+str(n+1)]=mapa_materias[str(mat)]["horas_estudio"] if mat != 0 else 0
      sample['HP'+str(n+1)]=mapa_materias[str(mat)]["horas_practica"] if mat != 0 else 0

    input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}
    predictions = modelo.predict(input_dict)
    prob = tf.nn.sigmoid(predictions[0])

    probabilidad_exito = prob.numpy()[0] * 100

    # Configurar el tamaño del gráfico
    fig, ax = plt.subplots(figsize=(3, 3))

    # Definir el ángulo de inicio y fin del arco de progreso
    start_angle = 90
    end_angle = 90 - (probabilidad_exito / 100 * 360)

    # Dibujar el arco exterior que representa el fondo de la barra de progreso
    outer_circle = plt.Circle((0, 0), 0.5, color='lightgrey', alpha=0.8)
    ax.add_artist(outer_circle)

    # Dibujar el arco que representa el progreso
    progress_arc = Wedge((0, 0), 0.5, end_angle, start_angle, color='lightgreen', alpha=0.8)
    ax.add_artist(progress_arc)

    # Añadir un círculo central para crear el efecto de "centro hueco"
    inner_circle = plt.Circle((0, 0), 0.3, color='white')
    ax.add_artist(inner_circle)

    # Calcular el porcentaje de progreso
    porcentaje = int(abs(end_angle - start_angle) / 360 * 100)

    # Agregar texto con el porcentaje en el centro del gráfico
    txt = ax.text(0, 0, f'{porcentaje}% de Éxito', ha='center', va='center', fontsize=15)
    txt._get_wrap_line_width = lambda : 50

    # Configurar los límites del eje
    ax.set_xlim(-0.5, 0.5)
    ax.set_ylim(-0.5, 0.5)

    # Quitar los ejes para un aspecto más limpio
    ax.axis('off')

    # Agrega el título al gráfico
    #ax.set_title('Probabilidad de Éxito',  fontsize=15)

    Referencias = ""

    if 0 <= probabilidad_exito <= 25:
      Referencias = "Análisis del Resultado: Es muy probable que recurses al menos una materia. Debés repensar todo el armado de tu cursada.\nNecesitás dedicarle mucho más tiempo a la facultad o reducir la cantidad de materias a cursar."
    elif  26 <= probabilidad_exito <= 50:
      Referencias = "Análisis del Resultado: Es probable que recurses al menos una materia. Deberias hacer algún cambio a tu cursada, como por ejemplo,\neliminar alguna materia, reemplazar materias exigentes por otras más sencillas o aumentar el tiempo que vas a dedicar a la facultad."
    elif  51 <= probabilidad_exito <= 75:
      Referencias = "Análisis del Resultado: Es probable que puedas aprobar o promocionar todas las materias, sin embargo, requerirá cierto grado de compromiso y dedicación de tu parte.\nPodés dejar la cursada tal como está y esforzarte durante todo el cuatrimestre o realizar alguna leve modificación en las materias, o en tu disponibilidad horaria, para aumentar esta probabilidad."
    else:
      Referencias = "Análisis del Resultado: Es muy probable que puedas aprobar o promocionar todas las materias. La cursada es acorde a las características ingresadas y no requiere ninguna modificación."

    return [fig, Referencias]

In [ ]:
with gr.Blocks(theme='ParityError/Interstellar') as app:
  gr.HTML("<center>"+ "<h1> Planificador de Materias</h1></center>")
  materias_pendiente_final = gr.Dropdown(choices=options, label="Seleccione Materias Pendiente Final", multiselect=True)
  with gr.Row():
    pdf_historia = gr.File(label="Subir PDF HistoriaAcademica", file_types=[".pdf"])
    pdf_oferta = gr.File(label="Subir PDF Oferta", file_types=[".pdf"])
  with gr.Row():
    with gr.Column():
      trabaja_input = gr.Checkbox(label="¿Trabaja?", value=False)
      ingresante_input = gr.Checkbox(label="¿Es Ingresante?", value=False)
    horas_input=gr.Number(value=1, minimum=1, maximum=168, step=1, label="Horas semanales",scale=6)
  plot_output = gr.Plot()
  b1 = gr.Button("Finalizar Carga")

  gr.HTML("<center>"+ "<h1>Probabilidad de éxito de una cursada</h1></center>")
  materias= gr.Dropdown(choices=options, label="Seleccione Materias", multiselect=True)
  with gr.Row():
    trabaja_input2 = gr.Checkbox(label="¿Trabaja?", value=False)
    horas_input2 = gr.Number(value=1, minimum=1, maximum=168, step=1, label="Horas semanales",scale=2)
  with gr.Row():
    prob_output = gr.Plot()
    prob_txt = gr.Textbox(label="Analisis",scale=2)

  b2 = gr.Button("Calcular Probabilidad")

  b1.click(main2, inputs=[materias_pendiente_final,pdf_oferta,trabaja_input,horas_input,ingresante_input,pdf_historia], outputs=plot_output)
  #b2.click(evaluar_exito_str, inputs=[materias,trabaja_input2,horas_input2], outputs=text)
  b2.click(evaluar_exito_str2, inputs=[materias,trabaja_input2,horas_input2], outputs=[prob_output,prob_txt])

app.launch(debug=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


themes/theme_schema@0.0.1.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://73ce1d429d0a6b953d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 521, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 276, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1945, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1513, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 807, in run
    r

1/1 [==============================] - 0s 47ms/step
